In [22]:
import warnings
warnings.filterwarnings('ignore')


import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime
import glob

In [20]:
ds_ref = xr.open_dataset('/N/project/Zli_lab/gongg/regrid/ref_.1deg.nc')

lat_min = float(ds_ref.lat.min())
lat_max = float(ds_ref.lat.max())
lon_min = float(ds_ref.lon.min())
lon_max = float(ds_ref.lon.max())


In [40]:
paths = [
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_06*',
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_07*',
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_08*',
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_09*'
]

all_files = []

for pattern in paths:
    files = glob.glob(pattern)
    all_files.extend(files)
all_files.sort()

In [8]:
ds_era = xr.open_dataset('/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_122022.nc')

In [14]:


ds_era_sorted = ds_era.sortby("latitude")
ds_era_sub = ds_era_sorted.sel(
    latitude=slice(lat_min, lat_max),
    longitude=slice(lon_min, lon_max)
)

ds_era_sub = ds_era_sub.rename({"latitude": "lat", "longitude": "lon"})
ds_era_sub["tp"] = ds_era_sub["tp"] * 1000
mask_nan = ds_ref["tp"].isel(time=0).isnull()
ds_era_sub = ds_era_sub.where(~mask_nan)


<xarray.Dataset> Size: 863MB
Dimensions:  (time: 744, lat: 250, lon: 580)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2022-12-01 ... 2022-12-31T23:00:00
  * lon      (lon) float32 2kB -124.8 -124.7 -124.6 -124.5 ... -67.1 -67.0 -66.9
  * lat      (lat) float32 1kB 24.4 24.5 24.6 24.7 24.8 ... 49.0 49.1 49.2 49.3
Data variables:
    tp       (time, lat, lon) float64 863MB nan nan nan nan ... nan nan nan nan


### do not run this again!

In [42]:
import glob
import xarray as xr
import numpy as np
from datetime import datetime

# ---------------------------
# 1. 利用 glob 获取所有目标文件并排序
# ---------------------------
patterns = [
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_06*',
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_07*',
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_08*',
    '/N/project/Zli_lab/amulla/JJA/ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_09*'
]

all_files = []
for pattern in patterns:
    all_files.extend(glob.glob(pattern))
all_files.sort()

# ---------------------------
# 2. 加载 ds_ref，用于获取裁剪区域与生成掩膜
# ---------------------------
# 请根据实际情况修改 ds_ref 的文件路径
ds_ref = xr.open_dataset('/N/project/Zli_lab/gongg/regrid/ref_.1deg.nc')
lat_min = float(ds_ref.lat.min())
lat_max = float(ds_ref.lat.max())
lon_min = float(ds_ref.lon.min())
lon_max = float(ds_ref.lon.max())
mask_nan = ds_ref["tp"].isel(time=0).isnull()

# 输出文件存储目录
output_dir = "/N/project/Zli_lab/gongg/ERA_data/LST"

# ---------------------------
# 3. 定义经度区间及对应的 UTC 偏移量
# ---------------------------
lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
utc_offsets = [-8, -7, -6, -5]

# ---------------------------
# 4. 遍历每个 ds_era 文件并处理
# ---------------------------
for i, file in enumerate(all_files, start=1):
    ds_era = xr.open_dataset(file)
    ds_era_sorted = ds_era.sortby("latitude")
    
    ds_era_sub = ds_era_sorted.sel(
        latitude=slice(lat_min, lat_max),
        longitude=slice(lon_min, lon_max)
    )
    ds_era_sub = ds_era_sub.rename({"latitude": "lat", "longitude": "lon"})
    ds_era_sub["tp"] = ds_era_sub["tp"] * 1000
    ds_era_sub = ds_era_sub.where(~mask_nan)
    
    # 提取年月（yyyymm），这里取 time 坐标中第一个时间点
    yyyymm = ds_era_sub.time.dt.strftime('%Y%m').values[0]
    
    for (lon_min_range, lon_max_range), offset in zip(lon_ranges, utc_offsets):
        mask = (ds_era_sub.lon >= lon_min_range) & (ds_era_sub.lon < lon_max_range)
        ds_lon_subset = ds_era_sub.where(mask, drop=True)
        if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            original_times = ds_lon_subset.time
            adjusted_times = original_times + np.timedelta64(offset, 'h')
            ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
            output_filename = f"{yyyymm}_U-{abs(offset)}.nc"
            output_path = f"{output_dir}/{output_filename}"
            ds_lon_subset.to_netcdf(output_path)
    
    # 每处理10个文件，打印一次当前时间
    if i % 10 == 0:
        print(datetime.now())


2025-03-26 22:09:08.296707
2025-03-26 22:09:39.035494
2025-03-26 22:10:08.740201
2025-03-26 22:10:41.006497
2025-03-26 22:11:13.440162
2025-03-26 22:11:45.754505
2025-03-26 22:12:18.746613
2025-03-26 22:12:52.638568
2025-03-26 22:13:25.778202
2025-03-26 22:13:59.197229
2025-03-26 22:14:33.834998
2025-03-26 22:15:07.132316
2025-03-26 22:15:40.927022
2025-03-26 22:16:13.587624
2025-03-26 22:16:46.063476
2025-03-26 22:17:19.839924
2025-03-26 22:18:09.446278
2025-03-26 22:19:00.115666


In [58]:
import os
import glob
import xarray as xr
from datetime import datetime

# 输入与输出目录
input_dir = "/N/project/Zli_lab/gongg/ERA_data/LST"
output_dir = "/N/project/Zli_lab/gongg/ERA_data/JJA_LST"

# 获取输入目录下所有 .nc 文件
all_files = glob.glob(os.path.join(input_dir, "*.nc"))

# 根据文件名按 (year, utc_offset) 分组
groups = {}
for file in all_files:
    basename = os.path.basename(file)  # 如 "200506_U-8.nc"
    name, ext = os.path.splitext(basename)  # name: "200506_U-8"
    # year 为前4位
    year = name[:4]
    # utc 部分：通过下划线分隔，第二部分如 "U-8"
    parts = name.split('_')
    if len(parts) < 2:
        continue
    utc_str = parts[1]  # e.g., "U-8"
    # 去掉 'U' 后转成 int（这里 -8 表示 UTC-8）
    offset = int(utc_str.replace('U', ''))
    key = (year, offset)
    groups.setdefault(key, []).append(file)

# 遍历每个分组，读取组合数据，筛选6、7、8月，并保存到指定目录
counter = 0
for (year, offset), files in groups.items():
    ds = xr.open_mfdataset(files, combine='by_coords')
    ds_jja = ds.where(ds.time.dt.month.isin([6, 7, 8]), drop=True)
    
    output_filename = f"{year}_U_{abs(offset)}.nc"
    output_path = os.path.join(output_dir, output_filename)
    ds_jja.to_netcdf(output_path)
    
    counter += 1
    if counter % 10 == 0:
        print(datetime.now())


2025-03-26 22:41:25.692688
2025-03-26 22:41:50.814597
2025-03-26 22:42:18.357485
2025-03-26 22:42:47.333969
2025-03-26 22:43:14.678724
2025-03-26 22:43:43.273382
2025-03-26 22:44:10.964123
2025-03-26 22:44:39.239085
2025-03-26 22:45:07.888390
2025-03-26 22:45:35.636872
2025-03-26 22:46:03.882590
2025-03-26 22:46:32.585437
2025-03-26 22:47:00.824420
2025-03-26 22:47:27.611738
2025-03-26 22:47:54.504160
2025-03-26 22:48:22.290772
2025-03-26 22:48:52.018127
2025-03-26 22:49:18.569681
